# 1. 读取配置文件

In [1]:
import json

In [2]:
config_path = "../exps/momentum.json"
with open(config_path, 'r') as f:
    args = json.load(f)
print(args)

{'prefix': 'train', 'dataset': 'cifar100', 'memory_size': 0, 'shuffle': True, 'init_cls': 50, 'increment': 10, 'model_name': 'momentum', 'convnet_type': 'resnet18', 'device': ['0'], 'seed': [1993], 'init_epochs': 250, 'init_lr': 0.1, 'init_weight_decay': 0.0005, 'epochs': 50, 'lr': 0.05, 'weight_decay': 0.0005, 'batch_size': 128, 'num_workers': 8, 'pin_memory': True, 'T': 2, 'pre_norm': True, 'momentum': 0.99, 'generator': 'noise'}


In [ ]:
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 2. 读取数据集

这里使用测试集进行分析

In [4]:
import torchvision as tv
from torch.utils.data import DataLoader

In [9]:
trsf = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std=(0.2675, 0.2565, 0.2761))
])
dataset = tv.datasets.CIFAR100(root='../data/CIFAR', train=False, transform=trsf, download=False)
dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=0)

In [11]:
len(dataset)

10000

# 2.1 tools

In [ ]:
from torch.nn import functional as F
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
def layernorm(x, eps=1e-05):
    """
    :param x: Tensor of [n, dim]
    """
    _mean = torch.mean(x, dim=1, keepdim=True)
    _std = torch.std(x, dim=1, keepdim=True)
    x = (x - _mean) / (_std + eps)
    return x

In [ ]:
def extract_features(extractor, loader):
    _features = []
    _labels = []
    extractor.to(device)
    extractor.eval()
    for _i, (_x, _y) in tqdm(enumerate(loader)):
        _x = _x.to(device)
        with torch.no_grad():
            _f = extractor(_x)["features"]
        _features.append(_f.to("cpu"))
        _labels.append(_y)
    _features = torch.cat(_features, dim=0)
    _labels = torch.cat(_labels, dim=0)
    return _features, _labels

In [ ]:
def show_tsne(features, labels):
    """
    :param features: ndarray of [n, feature_dim]
    :param labels: ndarray of [n]
    """
    _tsne = TSNE(n_components=2, init='pca', random_state=0)
    features = _tsne.fit_transform(features)
    plt.figure(figsize=(10, 10))
    plt.scatter(features[:, 0], features[:, 1], c=labels, cmap=plt.cm.get_cmap("jet", 100))
    plt.colorbar(ticks=range(100))
    plt.clim(-0.5, 9.5)
    plt.show()

# 3. 读取模型

In [ ]:
# 训练时不使用norm
saved_path = "../saved/noLN_momentum_resnet18_cifar100_50_all.pkl"
# 训练时使用norm
# saved_path = "../saved/e250+50_momentum_resnet18_cifar100_50_all.pkl"

In [3]:
import torch
from tqdm import tqdm
from utils.inc_net import HeadNet

In [ ]:
model = HeadNet(args, False, None)

In [ ]:
saved_all = torch.load(saved_path)
model.load_state_dict(saved_all["model_state_dict"], strict=False)
means = saved_all["means"]

backbone = model.backbone

## 3.1 计算每个类的特征向量

In [ ]:
features, labels = extract_features(backbone, dataloader)

## 3.2 对特征进行t-SNE可视化

In [ ]:
tsne = TSNE(n_components=2, init='pca', random_state=0)
features = tsne.fit_transform(features)

## 3.3 对norm后的特征进行t-SNE可视化

In [ ]:
features = layernorm(features)

array([[1, 2],
       [2, 2]])

In [ ]:
# self._predicts = np.stack((y_pred[:, 0], y_true), axis=1)